In [5]:
import argparse
import os
import numpy as np
import pickle
from importlib import reload

from analysis import forgetting

from wildtime import baseline_trainer, dataloader
from wildtime.configs.eval_fix import (
    configs_mimic_mortality, configs_mimic_readmission, 
    configs_fmow, configs_arxiv, configs_huffpost, configs_yearbook
)

`fused_weight_gradient_mlp_cuda` module not found. gradient accumulation fusion with weight gradient computation disabled.


In [45]:
reload(forgetting)

<module 'analysis.forgetting' from '/home/jaredfer/Projects/distshift/Wild-Time/analysis/forgetting.py'>

In [25]:
DATA_DIR="/data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results"

EXP_PATHS =  
    ('fmow', 'cs'): "0504_fmow_swa_coldstart_csaw",
    ('fmow', 'ws'): "0504_fmow_swa_csaw_lambda0.75", 
    ('huffpost', 'cs'): "0504_huffpost_swa_coldstart_csaw",
    ('huffpost', 'ws'): "0504_huffpost_swa_warmstart_csaw_2",
    ('arxiv', 'cs'): "0509_arxiv_swa_coldstart_csaw_seed_1" ,
    ('arxiv', 'ws'): "0509_arxiv_swa_warmstart_csaw_seed_1",
    ('yearbook', 'cs'): "0504_yearbook_swa_coldstart_csaw",
    ('yearbook', 'ws'): "0504_yearbook_swa_csaw_lambda0.75",
}

OPTIMAL_DECAY = {
    ('fmow', 'cs'): 0.90,
    ('fmow', 'ws'): 0.75, 
    ('huffpost', 'cs'): 0.85,
    ('huffpost', 'ws'): 0.45,
    ('arxiv', 'cs'): 0.75,
    ('arxiv', 'ws'): 0.50,
    ('yearbook', 'cs'): 0.90,
    ('yearbook', 'ws'): 0.75,
}

In [6]:
EXP_PARAMS = {
    'dataset': "huffpost",
    'method': 'swa',
    'eval_batch_size': 2048,
    'swa_ewa_lambda': 0.50,
    'exp_path': '0124_huffpost_agem_agem_0',

    'device': 0,
    'random_seed': 1,
    'num_workers': 0,

    'eval_fix': True,
    'eval_warmstart_finetune': False,
    'eval_features': False,
    'linear_probe': False,
    'online': False,
    'offline_steps': 1000,
    'online_steps': 1000,

    'eval_all_timestamps': False,

    'load_model': False,
    'torch_compile': False,
    'sam': False,
    'swa_ewa': True,
    'swa_steps': None,
    'swa_load_from_checkpoint': True,

    'data_dir': '/data/tir/projects/tir6/strubell/data/wilds/data',
    'log_dir': '/data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results',
    'results_dir': '/data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/icml/huffpost/agem',
    'checkpoint_path': ''

    # 'checkpoint_path': '/data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/0509_arxiv_swa_warmstart_csaw_seed_1/time_2016'
}

if EXP_PARAMS['dataset'] == 'fmow':
    config =  {**configs_fmow.configs_fmow_erm, **EXP_PARAMS}
    eval_checkpoints = [f"time_{i}.pth" for i in range(11)]
if EXP_PARAMS['dataset'] == 'huffpost':
    config =  {**configs_huffpost.configs_huffpost_erm, **EXP_PARAMS}
    eval_checkpoints = [f"time_{i}.pth" for i in range(2012, 2016)]
if EXP_PARAMS['dataset'] == 'arxiv':
    eval_checkpoints = [f"time_{i}.pth" for i in range(2007, 2016)]
    config = {**configs_arxiv.configs_arxiv_erm, **EXP_PARAMS}
if EXP_PARAMS['dataset'] == 'yearbook':
    eval_checkpoints = [f"time_{i}.pth" for i in range(1930, 1970)]
    config = {**configs_yearbook.configs_yearbook_erm, **EXP_PARAMS}
if EXP_PARAMS['dataset'] == 'mimic':
    config = {**configs_mimic_mortality.configs_mimic_mortality_erm, **EXP_PARAMS}

experimental_config = argparse.Namespace(**config)

In [7]:
trainer = baseline_trainer.init(experimental_config)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertFeaturizer: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertFeaturizer from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertFeaturizer from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import forgetting_main as fm
reload(fm)

trainer.swa_ewa_lambda = 0.50

eval_ckpts = [f"time_{i}.pth" for i in range(2012, 2016)]
data_dir = os.path.join(config['results_dir'], config['exp_path'])
_, pred_logits, labels = fm.get_predictions(trainer,  [f"time_{i}.pth" for i in range(2012, 2016)], data_dir)


Loading time_2012.pth


  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [8]:
for i in [0, 0.5, 1]:
    trainer.create_csaw_from_ckpts(0, lambda_val=i)
    print()
    # print(trainer.swa_model.module.classifier.bias)

Updated model from /data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/icml/huffpost/agem/0124_huffpost_agem_agem_0/checkpoints/time_2012.pth
Parameter containing:
tensor([ 0.0326, -0.0031,  0.0134, -0.0173,  0.0079,  0.0281, -0.0249, -0.0058,
         0.0081, -0.0186,  0.0353], device='cuda:0', requires_grad=True)
Updated model from /data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/icml/huffpost/agem/0124_huffpost_agem_agem_0/checkpoints/time_2013.pth
Parameter containing:
tensor([ 0.0326, -0.0031,  0.0134, -0.0173,  0.0079,  0.0281, -0.0249, -0.0058,
         0.0081, -0.0186,  0.0353], device='cuda:0', requires_grad=True)
Updated model from /data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/icml/huffpost/agem/0124_huffpost_agem_agem_0/checkpoints/time_2014.pth
Parameter containing:
tensor([ 0.0326, -0.0031,  0.0134, -0.0173,  0.0079,  0.0281, -0.0249, -0.0058,
         0.0081, -0.0186,  0.0353], device='cuda:0', requires_grad=True

In [ ]:
trainer.create_csaw_from_ckpts(0, 1)
trainer.swa_model.module.classifier.bias

In [16]:
trainer.create_csaw_from_ckpts(0, 0.85)

eval_splits = [t for t in trainer.eval_dataset.ENV if t > trainer.split_time]
accs = []
for t in eval_splits:
   accs.append(trainer.run_eval_timestamp(t, 1)[0])
   print(t, accs[-1])

Updated model from /data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/icml/huffpost/agem/0124_huffpost_agem_agem_0/checkpoints/time_2012.pth
Parameter containing:
tensor([ 0.0326, -0.0031,  0.0134, -0.0173,  0.0079,  0.0281, -0.0249, -0.0058,
         0.0081, -0.0186,  0.0353], device='cuda:0', requires_grad=True)
Updated model from /data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/icml/huffpost/agem/0124_huffpost_agem_agem_0/checkpoints/time_2013.pth
Parameter containing:
tensor([ 0.0293,  0.0047,  0.0152, -0.0298,  0.0089,  0.0331, -0.0329, -0.0180,
        -0.0021, -0.0255,  0.0450], device='cuda:0', requires_grad=True)
Updated model from /data/tir/projects/tir6/strubell/jaredfer/projects/wild-time/results/icml/huffpost/agem/0124_huffpost_agem_agem_0/checkpoints/time_2014.pth
Parameter containing:
tensor([ 0.0227,  0.0085,  0.0207, -0.0399,  0.0098,  0.0433, -0.0276, -0.0325,
        -0.0032, -0.0379,  0.0546], device='cuda:0', requires_grad=True

100%|██████████| 1/1 [00:02<00:00,  2.78s/it]


2016 0.4044368600682594


100%|██████████| 1/1 [00:02<00:00,  2.09s/it]


2017 0.4510250569476082


100%|██████████| 1/1 [00:00<00:00,  1.06it/s]

2018 0.47680412371134023


In [15]:
import logging
print(accs, np.mean(accs))

[0.39334470989761094, 0.4419134396355353, 0.47164948453608246] 0.43563587802307624


In [57]:
dataset = "arxiv"
init = "ws"

EXP_DIR = EXP_PATHS[(dataset, init)]

data_fpath = os.path.join(DATA_DIR, EXP_DIR, f'preds_meta_swa_{OPTIMAL_DECAY[(dataset, init)]}.pkl')
data_swa = forgetting.load_predictions_and_metadata(data_fpath)


data_fpath = os.path.join(DATA_DIR, EXP_DIR, 'preds_meta_swa_1.0.pkl')
data = forgetting.load_predictions_and_metadata(data_fpath)

In [ ]:
# FT WS
[_[-1].mean() for _ in corr], [_[-1].mean() for _ in corr_swa]

In [ ]:
[_[-1].mean() for _ in corr], [_[-1].mean() for _ in corr_swa]

In [ ]:
[[_, corr[-1][_].mean()] for _ in range(11)]

In [58]:
print(np.mean([_[-1].mean() for _ in corr]))
print(np.mean([_[-1].mean() for _ in corr_swa]))




0.5363446713363825
0.5417760540236131


In [60]:
corr

[array([[False, False, False, ..., False, False,  True],
        [False,  True, False, ...,  True, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [False,  True, False, ..., False,  True, False],
        [ True, False,  True, ..., False, False, False],
        [ True,  True,  True, ...,  True,  True,  True]]),
 array([[ True,  True, False, ..., False, False, False],
        [False, False,  True, ..., False, False,  True],
        [False, False,  True, ..., False,  True,  True],
        ...,
        [False,  True, False, ...,  True,  True,  True],
        [False,  True,  True, ..., False,  True,  True],
        [False, False, False, ...,  True,  True, False]]),
 array([[False, False,  True, ..., False, False, False],
        [False,  True, False, ..., False, False,  True],
        [False,  True,  True, ..., False,  True,  True],
        ...,
        [ True, False,  True, ..., False, False,  True],
        [False,  True, False, ..., False, Fal

In [56]:
corr = forgetting.get_correctness(data)
corr_swa = forgetting.get_correctness(data_swa)

ft_events = forgetting.evaluate_method(corr, corr, diff_all=False)
csaw_events = forgetting.evaluate_method(corr, corr_swa, diff_all=False)

for t in range(len(ft_events[0])):
    print(f"Eval on {t}")
    print("Transferred", np.array(csaw_events[0][t]) - np.array(ft_events[0][t]))
    print("Hard", np.array(csaw_events[1][t]) - np.array(ft_events[1][t]))
    print("Learned", np.array(csaw_events[2][t]) - np.array(ft_events[2][t]))
    print("Forgotten", np.array(csaw_events[3][t]) - np.array(ft_events[3][t]), '\n')


Eval on 0
Transferred [  97   75 -168  -17   -3   13   -2   19   19]
Hard [-135 -158  220   49   24  -22    3  -18  -16]
Learned [ 135  158 -220  -49  -24   22   -3   18   16]
Forgotten [-97 -75 168  17   3 -13   2 -19 -19] 

Eval on 1
Transferred [  94   95 -159  -32  -19    4   -1   35   54]
Hard [-200 -173  255   77    6    2   21   -9  -15]
Learned [ 200  173 -255  -77   -6   -2  -21    9   15]
Forgotten [-94 -95 159  32  19  -4   1 -35 -54] 

Eval on 2
Transferred [ 113  128 -201  -21    7   40   -9   24   69]
Hard [-178 -246  277   53  -19    0   16  -59  -24]
Learned [ 178  246 -277  -53   19    0  -16   59   24]
Forgotten [-113 -128  201   21   -7  -40    9  -24  -69] 

Eval on 3
Transferred [ 197  114 -144  -23   23   34  -32   37   71]
Hard [-272 -260  324    1   11  -64   -7  -40  -40]
Learned [ 272  260 -324   -1  -11   64    7   40   40]
Forgotten [-197 -114  144   23  -23  -34   32  -37  -71] 

Eval on 4
Transferred [ 253  182 -195  -68   17   80   13   36   71]
Hard [-33

In [ ]:
sankey_nodes = construct_example_flow_graph(data_swa['transferred'][-1][6:], data_swa['hard'][-1][6:], data_swa['learned'][-1][6:], data_swa['forgotten'][-1][6:])
source, target,value, node_labels, color, node_heights = convert_flow_graph_to_plotly(sankey_nodes)
plot_sankey(source, target,value, node_labels, color, node_heights / max(node_heights), title=f"CSAW: {data['model']}").show()

In [ ]:
sankey_nodes = construct_example_flow_graph(data['transferred'][-1][6:], data['hard'][-1][6:], data['learned'][-1][6:], data['forgotten'][-1][6:])
source, target,value, node_labels, color, node_heights = convert_flow_graph_to_plotly(sankey_nodes)
plot_sankey(source, target,value, node_labels, color, node_heights / max(node_heights), title=f"FT: {data['model']}").show()

In [ ]:
sankey_nodes = construct_example_flow_graph(data['transferred'][-1][8:], data['hard'][-1][8:], data['learned'][-1][8:], data['forgotten'][-1][8:])
source, target,value, node_labels, color, node_heights = convert_flow_graph_to_plotly(sankey_nodes)
plot_sankey(source, target,value, node_labels, color, node_heights / max(node_heights), title=f"Splits 11: {data['model']}").show()

In [ ]:

transferred, hard, learned, forgotten = find_learning_and_forgetting_events(np.hstack(labels), pred_labels=np.hstack(pred_labels))

sankey_nodes = construct_example_flow_graph(transferred, hard, learned, forgotten)

# source, target,value, labels, color, node_heights
source, target,value, node_labels, color, node_heights = convert_flow_graph_to_plotly(
    sankey_nodes, learned, forgotten
)
plot_sankey([], color, source, target, value, title=f"All Eval Splits: {MODEL}").show()

In [ ]:
MODEL = "Warmstart FT"
for split, label, pred_label in zip(eval_splits, labels, pred_labels):
    transferred, hard, learned, forgotten = find_learning_and_forgetting_events(label, pred_labels=pred_label)

    sankey_nodes = construct_example_flow_graph(transferred, hard, learned, forgotten)
    source, target,value, node_labels, color, node_heights = convert_flow_graph_to_plotly(sankey_nodes)
    plot_sankey([], color, source, target, value, title=f"at time {split}: {MODEL}").show()